## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.

In [ ]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import seaborn as sns
import os

import pinon as pn
import pinon.names as pn_cols
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts

config = pn.Config('master', 'V', 10)

In [22]:
config.get_target_tickers()

KeyboardInterrupt: 

In [20]:
c = pn.Comps(config)
c.run()

Adding breaking report for ticker V for report date 2023-03-31
Adding breaking report for ticker DFS for report date 2023-03-31


KeyboardInterrupt: 

In [ ]:
c.target_ks.index.to_list()

In [ ]:
dp = pn.DailyPrices()
dp.get_downsampled_share_price_ratios('V')[sf_cols.PE_TTM].tail(70).plot()

In [ ]:
c.m.multiples.loc[('AXP',), pn_cols.TTM_PE_RATIO].plot()

In [ ]:
c.m.multiples.loc[('V', '2009-12-31:2010-06-30')]